<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#RNN-y-LSTM" data-toc-modified-id="RNN-y-LSTM-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>RNN y LSTM</a></span><ul class="toc-item"><li><span><a href="#Ejemplo-análisis-de-sentimiento-con-dataset-IMDB" data-toc-modified-id="Ejemplo-análisis-de-sentimiento-con-dataset-IMDB-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Ejemplo análisis de sentimiento con dataset IMDB</a></span></li><li><span><a href="#Ejemplo-Generación-de-texto" data-toc-modified-id="Ejemplo-Generación-de-texto-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Ejemplo Generación de texto</a></span></li></ul></li></ul></div>

# RNN y LSTM

## Ejemplo análisis de sentimiento con dataset IMDB 

In [1]:
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [55]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

In [102]:
# hacemos que todas las entradas tengan la misma longitud = 150
X_train = pad_sequences(X_train, maxlen=150)
X_test = pad_sequences(X_test, maxlen=150)

In [103]:
X_train.shape, X_test.shape

((25000, 150), (25000, 150))

In [112]:
palabra_a_id = keras.datasets.imdb.get_word_index()
id_a_palabra = {i: palabra for palabra, i in palabra_a_id.items()}
print('Comentario: ' + str( [id_a_palabra.get(i, ' ') for i in X_train[6]]) )
print('Etiqueta: ' + str(y_train[6]))

Comentario: [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'of', 'props', 'this', 'and', 'concentrates', 'concept', 'issue', 'skeptical', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'and', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'starship', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'originals', 'things', 'is', 'far', 'this', 'make', 'mistakes', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'dose', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its

In [121]:
model = keras.models.Sequential([
        keras.layers.Embedding(input_dim=10000 , output_dim=32, input_length=150),
        keras.layers.LSTM(32, dropout=0.1 ,recurrent_dropout=0.2, return_sequences=True),
        keras.layers.LSTM(32, dropout=0.1 ,recurrent_dropout=0.2),
        keras.layers.Dense(1, activation='sigmoid')
])

In [123]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 150, 32)           320000    
_________________________________________________________________
lstm_9 (LSTM)                (None, 150, 32)           8320      
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 336,673
Trainable params: 336,673
Non-trainable params: 0
_________________________________________________________________


In [124]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [125]:
model.fit(X_train,y_train, epochs=5, batch_size=64, verbose=1, validation_data=(X_test,y_test))

Epoch 1/5
391/391 [==============================] - 126s 323ms/step - loss: 0.4405 - acc: 0.7894 - val_loss: 0.4044 - val_acc: 0.8386
Epoch 2/5
391/391 [==============================] - 132s 337ms/step - loss: 0.2837 - acc: 0.8856 - val_loss: 0.3560 - val_acc: 0.8409
Epoch 3/5
391/391 [==============================] - 137s 350ms/step - loss: 0.2334 - acc: 0.9082 - val_loss: 0.3724 - val_acc: 0.8421
Epoch 4/5
391/391 [==============================] - 132s 338ms/step - loss: 0.1912 - acc: 0.9285 - val_loss: 0.4415 - val_acc: 0.8373
Epoch 5/5
391/391 [==============================] - 137s 350ms/step - loss: 0.1570 - acc: 0.9431 - val_loss: 0.4201 - val_acc: 0.8315


In [126]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Exactitud: {:.2f}'.format(scores[1]))

Exactitud: 0.83


In [189]:
texto_neg = X_test[9]
texto_pos = X_test[13]
texts = (texto_neg, texto_pos)
textos = pad_sequences(texts, maxlen=300, value = 0.0) 
preds = model.predict(textos)
print("predicciones:" , preds)

predicciones: [[0.97783935]
 [0.00336328]]


In [190]:
from nltk import word_tokenize
from keras.preprocessing import sequence

test=[]
for t in word_tokenize("had wont only second for from subject"):
     test.append(palabra_a_id[t])

test=sequence.pad_sequences([test],maxlen=150)
r = model.predict(test)
print(r)
if(r < 0.5):
    print("negativo")
else:
    print("positivo")


[[0.79456925]]
positivo


## Ejemplo Generación de texto

In [2]:
import numpy as np
import pandas as pd
import string
import tensorflow as tf

path = tf.keras.utils.get_file('Shakespear.txt', 'https://www.gutenberg.org/files/55206/55206-0.txt')
text = open(path, 'rb').read().decode(encoding='utf-8')
print(text[:500])

﻿The Project Gutenberg EBook of Fábulas, by Félix Samaniego

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Fábulas

Author: Félix Samaniego

Release Date: July 26, 2017 [EBook #55206]

Language: Spanish

Character set encoding: UTF-8

*** START OF THIS PROJECT G


In [249]:
def clean_doc(doc):
    clean_words= []
    text = doc.replace("-"," ")
    text = text.replace("--"," ")
    text = text.replace(","," ")
    text = text.replace("."," ")
    text = text.replace("/","") 
    text = text.replace("\n"," ") 
    text = text.replace("\t"," ") 
    text = text.replace("'","") 
    
    words = word_tokenize(text)
    for word in words:
        word = word.strip(string.punctuation)
        if len(word)>=1 and word.isdigit()==False:
                word = word.lower()
                clean_words.append(word)
    return clean_words


In [250]:
text = clean_doc(text)

In [3]:
print('Longitud del texto: {} carácteres'.format(len(text)))
vocab = sorted(set(text))
print ('El texto está compuesto de:  {} carácteres'.format(len(vocab)))


Longitud del texto: 326632 carácteres
El texto está compuesto de:  115 carácteres


In [30]:
# creamos un índice para cada caracter del texto y luego creamos secuencias
#de tamaño 100 con estos indices
char_a_ind = {char:i for i, char in enumerate(vocab)}
ind_a_char = np.array(vocab)
codif_text = np.array([char_a_ind[c] for c in text])

long_seq = 100
dataset_caract = tf.data.Dataset.from_tensor_slices(codif_text)
secuencias = dataset_caract.batch(long_seq+1, drop_remainder=True)
for i in secuencias.take(2):
    print(ind_a_char[i.numpy()])

['\ufeff' 'T' 'h' 'e' ' ' 'P' 'r' 'o' 'j' 'e' 'c' 't' ' ' 'G' 'u' 't' 'e'
 'n' 'b' 'e' 'r' 'g' ' ' 'E' 'B' 'o' 'o' 'k' ' ' 'o' 'f' ' ' 'F' 'á' 'b'
 'u' 'l' 'a' 's' ',' ' ' 'b' 'y' ' ' 'F' 'é' 'l' 'i' 'x' ' ' 'S' 'a' 'm'
 'a' 'n' 'i' 'e' 'g' 'o' '\r' '\n' '\r' '\n' 'T' 'h' 'i' 's' ' ' 'e' 'B'
 'o' 'o' 'k' ' ' 'i' 's' ' ' 'f' 'o' 'r' ' ' 't' 'h' 'e' ' ' 'u' 's' 'e'
 ' ' 'o' 'f' ' ' 'a' 'n' 'y' 'o' 'n' 'e' ' ' 'a' 'n']
['y' 'w' 'h' 'e' 'r' 'e' ' ' 'a' 't' ' ' 'n' 'o' ' ' 'c' 'o' 's' 't' ' '
 'a' 'n' 'd' ' ' 'w' 'i' 't' 'h' '\r' '\n' 'a' 'l' 'm' 'o' 's' 't' ' ' 'n'
 'o' ' ' 'r' 'e' 's' 't' 'r' 'i' 'c' 't' 'i' 'o' 'n' 's' ' ' 'w' 'h' 'a'
 't' 's' 'o' 'e' 'v' 'e' 'r' '.' ' ' ' ' 'Y' 'o' 'u' ' ' 'm' 'a' 'y' ' '
 'c' 'o' 'p' 'y' ' ' 'i' 't' ',' ' ' 'g' 'i' 'v' 'e' ' ' 'i' 't' ' ' 'a'
 'w' 'a' 'y' ' ' 'o' 'r' '\r' '\n' 'r' 'e' '-']


In [6]:
def separa_entrada_etiqueta(p):
    texto_entrada = p[:-1]
    text_destino = p[1:]
    return texto_entrada, text_destino

dataset = secuencias.map(separa_entrada_etiqueta)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int32, tf.int32)>

In [7]:
TAM_LOTE = 64
TAM_BUFFER = 10000

dataset = dataset.shuffle(TAM_BUFFER).batch(TAM_LOTE, drop_remainder=True)
print (dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>


In [8]:
tam_vocab = len(vocab)
tam_embedding = 256
unidades_rnn = 1024

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def crear_modelo(tam_vocab, tam_embedding, unidades_rnn, tam_lote):
    model = Sequential()
    model.add(Embedding(input_dim=tam_vocab, 
                      output_dim=tam_embedding, 
                      batch_input_shape=[tam_lote, None]))
    model.add(LSTM(unidades_rnn,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
    model.add(Dense(tam_vocab))
    return model

In [10]:
model = crear_modelo(
      tam_vocab = len(vocab),
      tam_embedding=tam_embedding,
      unidades_rnn=unidades_rnn,
      tam_lote=TAM_LOTE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           29440     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 115)           117875    
Total params: 5,394,291
Trainable params: 5,394,291
Non-trainable params: 0
_________________________________________________________________


In [12]:
def perdida(etiquetas, preds):
    return tf.keras.losses.sparse_categorical_crossentropy(etiquetas, preds, from_logits=True)

model.compile(optimizer='adam', loss=perdida)

epocas = 5
history = model.fit(dataset, epochs=epocas, verbose=1)

Epoch 1/5
50/50 [==============================] - 276s 6s/step - loss: 2.0987
Epoch 2/5
50/50 [==============================] - 281s 6s/step - loss: 1.9841
Epoch 3/5
50/50 [==============================] - 280s 6s/step - loss: 1.8907
Epoch 4/5
50/50 [==============================] - 291s 6s/step - loss: 1.8148
Epoch 5/5
50/50 [==============================] - 277s 6s/step - loss: 1.7468


In [16]:
from tensorflow.keras.models import load_model
model.save('modelo.h5') 

model = crear_modelo(tam_vocab, tam_embedding, unidades_rnn, tam_lote=1)
model.load_weights('modelo.h5')
model.build(tf.TensorShape([1, None]))

In [17]:
def generar_texto(model, cad_inicio):
    total_gen = 1000
    ent_e = [char_a_ind[s] for s in cad_inicio]
    ent_e = tf.expand_dims(ent_e, 0)
    texto_generado = []
    temperature = 0.5
    model.reset_states()
    for i in range(total_gen):
        predicciones = model(ent_e)      
        predicciones = tf.squeeze(predicciones, 0)
        predicciones = predicciones / temperature
        id_prediccion = tf.random.categorical(predicciones, num_samples=1)[-1,0].numpy()
        ent_e = tf.expand_dims([id_prediccion], 0)
        texto_generado.append(ind_a_char[id_prediccion])
    return (cad_inicio + ''.join(texto_generado))

In [29]:
print(generar_texto(model, cad_inicio=u"libro"))

libro,
    La valla de caltado el conseros
    En la astina con un calla bora atrajado de la nota contancia, como es en la cadan una espentinos el más estesdares que se una vera confera tande la antales, que pora de su vedar el perro á las abulas de un otro plos se tropectad the brejectr the the un the with
writererg-tm works andito que me con gratice lis prouided th the inderit ene the wiand of ent this fite suedo te andiguit this work ino the morted the Project Gutenberg-tm imprestion yous ardicion arditorion dis Projer work at of in el otration this como the Pores arditite in inditad the brecertion
                                                                                                                                                                                                                                                                                                                                                                                                       